In [66]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [67]:
from openai import OpenAI

client = OpenAI() # defautls to getting the key using os.environ.get("OPENAI_API_KEY")

In [68]:
# Upload LinkedIn profile to OpenAI
profile = client.files.create(
  file=open("LinkedIn_profile.json", "rb"),
  purpose="assistants"
)
print(profile)

FileObject(id='file-Sc3TiWy4VBnmyUH3hO8fov1d', bytes=4863, created_at=1721983353, filename='LinkedIn_profile.json', object='file', purpose='assistants', status='processed', status_details=None)


In [69]:
# Create Vector Store containing the LinkedIn profile
vector_store = client.beta.vector_stores.create(
  name="Example_User LinkedIn Profile",
  file_ids=[profile.id]
)
print(vector_store)

VectorStore(id='vs_2cR4eXl6jG0X0q2AFtszW0yh', created_at=1721983355, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=1, total=1), last_active_at=1721983355, metadata={}, name='Example_User LinkedIn Profile', object='vector_store', status='in_progress', usage_bytes=0, expires_after=None, expires_at=None)


In [70]:
# Attach the vectorized example linkedIn profile to this thread
with open("job_description.json", "r") as f:
    # read in contents of job description
    job_description = f.read()

thread = client.beta.threads.create(
    tool_resources={
        'file_search': {
            'vector_store_ids': [vector_store.id]
        }
    },
    messages=[
        {
            'role': 'user',
            'content': job_description
        }
    ]
)

print(thread)

Thread(id='thread_McvBEGxOhWPvR2o0wLJOnEys', created_at=1721983358, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_2cR4eXl6jG0X0q2AFtszW0yh'])))


In [71]:
import os

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=os.environ.get("ASSISTANT_ID")
)

print(run)

Run(id='run_MPL0YmKrW5DbOl8xjKe1Rw0D', assistant_id='asst_fixO3zdAF2VSLVasrSpNMrDt', cancelled_at=None, completed_at=None, created_at=1721983362, expires_at=1721983962, failed_at=None, incomplete_details=None, instructions='You are a Resume Building Expert! \n\nThe user will input a job description. You will use the user\'s attached JSON LinkedIn profile to build a resume that will give the user the best chance of getting the job described in the job description.\n\nThe created resume should be in JSON format, according to the following JSON Schema:\n{\n  "properties": {\n    "basics": {\n      "type": "object",\n      "additionalProperties": true,\n      "properties": {\n        "name": {\n          "type": "string"\n        },\n        "label": {\n          "type": "string",\n          "description": "e.g. Web Developer"\n        },\n        "image": {\n          "type": "string",\n          "description": "URL (as per RFC 3986) to a image in JPEG or PNG format"\n        },\n        

In [76]:
# Retreive all run steps
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

# Find message creation text step and extract content
text_creation_step = run_steps.data[0]
message_id = text_creation_step.step_details.message_creation.message_id

message = client.beta.threads.messages.retrieve(
  message_id=message_id,
  thread_id=thread.id,
)

message_text = message.content[0].text.value
print(message_text)

Based on the provided job description for a Software Engineer III position at Google, and information extracted from the user's LinkedIn profile, the following resume has been tailored to match the requirements and preferences of the role:

```json
{
  "basics": {
    "name": "Jane Doe",
    "label": "Senior Software Engineer",
    "email": "jane.doe@example.com",
    "phone": "(555) 123-4567",
    "url": "https://www.linkedin.com/in/janedoe",
    "summary": "Experienced Software Engineer with a solid background in developing scalable web applications and leading engineering teams. Skilled in JavaScript, React, Node.js, and Python with a strong foundation in data structures, algorithms, and system design. Passionate about leveraging technical expertise to develop innovative solutions that impact millions of users.",
    "location": {
      "address": "San Francisco, California, United States"
    },
    "profiles": [
      {
        "network": "LinkedIn",
        "username": "janedoe",

In [77]:
import json

# Extract the JSON resume from the message content
start = message_text.index("```json") + 7
end = message_text.index("```", start)

json_resume = json.loads(message_text[start:end])
print(json_resume)

{'basics': {'name': 'Jane Doe', 'label': 'Senior Software Engineer', 'email': 'jane.doe@example.com', 'phone': '(555) 123-4567', 'url': 'https://www.linkedin.com/in/janedoe', 'summary': 'Experienced Software Engineer with a solid background in developing scalable web applications and leading engineering teams. Skilled in JavaScript, React, Node.js, and Python with a strong foundation in data structures, algorithms, and system design. Passionate about leveraging technical expertise to develop innovative solutions that impact millions of users.', 'location': {'address': 'San Francisco, California, United States'}, 'profiles': [{'network': 'LinkedIn', 'username': 'janedoe', 'url': 'https://www.linkedin.com/in/janedoe'}]}, 'work': [{'name': 'TechCorp', 'location': 'San Francisco, CA', 'description': 'A leading technology firm specializing in web-based applications.', 'position': 'Senior Software Engineer', 'startDate': '2022-01', 'endDate': 'Present', 'summary': 'Lead the development and m